In [ ]:
!pip install transformers datasets scikit-learn pandas --quiet

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
yelp = pd.read_csv('/content/drive/My Drive/ml project/yelp.csv')
zomato = pd.read_csv('/content/drive/My Drive/ml project/zomato_reviews.csv')
rest = pd.read_csv('/content/drive/My Drive/ml project/restrev.csv')

In [ ]:
def clean_and_prepare(df, review_col, rating_col):
    df = df[[review_col, rating_col]].rename(columns={review_col: "Review", rating_col: "Rating"})
    df.dropna(inplace=True)
    df["Review"] = df["Review"].astype(str)
    df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
    return df.dropna()

In [ ]:
df = pd.concat([
    clean_and_prepare(yelp, "Review Text", "Rating"),
    clean_and_prepare(zomato, "review", "rating"),
    clean_and_prepare(rest, "Review", "Rating")])

In [ ]:
df["label"] = df["Rating"].apply(lambda x: 1 if x >= 4 else 0)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Review"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated key name
    save_strategy="epoch",  # Match with eval_strategy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
    learning_rate=2e-5,
    weight_decay=0.01,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-57-5117c6a9592a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
metrics = trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.378400,0.309122,0.860600,0.895157,0.925804,0.866474
2,0.209400,0.322272,0.870082,0.905556,0.904253,0.906862
3,0.243000,0.355948,0.865553,0.902203,0.901461,0.902947


In [ ]:
print("\n Final Metrics:")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key.replace('eval_', '').capitalize()}: {value * 100:.2f}%")


📊 Final Metrics:
Loss: 32.23%
Accuracy: 87.01%
F1: 90.56%
Precision: 90.43%
Recall: 90.69%
Runtime: 2805.24%
Samples_per_second: 25188.60%
Steps_per_second: 3151.20%


In [ ]:
save_directory = '/content/drive/My Drive/my_trained_model'
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Model saved to: {save_directory}")

Model saved to: /content/drive/My Drive/my_trained_model


In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


Device set to use cuda:0


In [ ]:
test_review = "food was bad!"
result = sentiment_pipeline(test_review)[0]
print("\n📝 Review:", test_review)
print("🔍 Sentiment:", result["label"], "with score", round(result["score"], 4))


📝 Review: food was bad!
🔍 Sentiment: LABEL_0 with score 0.9426
